In [1]:
%matplotlib inline
# Imports 
import tensorflow as tf
from keras.datasets import fashion_mnist
from keras.layers import Dense, Flatten, Dropout,MaxPooling2D,GlobalMaxPooling2D,UpSampling2D,Conv2DTranspose,Reshape
from keras.layers import Conv2D
from tensorflow.keras.utils import to_categorical,plot_model
from keras import Input , Model
import matplotlib.pyplot as plt

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [2]:
# Make input daat from 0 to 1
x_train = x_train.reshape(60000,28,28,1).astype("float32") / 255.
x_test = x_test.reshape(10000,28,28,1).astype("float32") / 255.

# convert class vectors to binary class matrices
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


def Encoder(input_shape = (28,28,1)):
    # Encoder 
    encoder_input = Input(shape=input_shape, name='original_img')
    x = Conv2D(16, 3, activation='relu')(encoder_input)
    x = Conv2D(32, 3, activation='relu')(x)
    x = MaxPooling2D(3)(x)
    x = Conv2D(32, 3, activation='relu')(x)
    x = Conv2D(16, 3, activation='relu')(x)
    encoder_output = GlobalMaxPooling2D()(x)

    encoder = Model(encoder_input, encoder_output, name='encoder')
    print(encoder.summary())
    return encoder

def Decoder(input_shape = (16,1)):
    # Decoder 
    decoder_input = Input(shape=input_shape, name='encoded_img')
    x = Reshape((4, 4, 1))(decoder_input)
    x = Conv2DTranspose(16, 3, activation='relu')(x)
    x = Conv2DTranspose(32, 3, activation='relu')(x)
    x = UpSampling2D(3)(x)
    x = Conv2DTranspose(16, 3, activation='relu')(x)
    decoder_output = Conv2DTranspose(1, 3, activation='relu')(x)

    decoder = Model(decoder_input, decoder_output, name='decoder')
    print(decoder.summary())
    return decoder

def AE_MNIST(input_shape = (28,28,1)):
  # Encoder 
  encoder = Encoder()

  # Decoder 
  decoder = Decoder()

  # Autoencoder
  autoencoder_input = Input(shape=input_shape, name='img')
  encoded_img = encoder(autoencoder_input)
  decoded_img = decoder(encoded_img)
  autoencoder = Model(autoencoder_input, decoded_img, name='autoencoder')
  autoencoder.compile(loss = 'mse', optimizer= 'sgd',metrics =['acc' , 'mse'])
  print(autoencoder.summary())
  
  return autoencoder


def train(model):
  with tf.device('/device:GPU:0'):
      history = model.fit(x_train,y_train ,
                          batch_size = 128,
                          epochs = 10,
                          verbose = 1,
                          validation_split = 0.2)
            
  return history.history

In [3]:
autoencoder = AE_MNIST()
train(autoencoder)

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
original_img (InputLayer)    [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 32)          9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 16)          4624      
_________________________________________________________________
global_max_pooling2d (Global (None, 16)                0   

ValueError: ignored